### Imports

In [ ]:
import pathlib
import random
import time

import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torch_geometric.utils as tg_utils
from tqdm.notebook import tqdm, trange
from MIDS_dataset import MIDSDataset, MIDSLabelsDataset, MIDSProbabilitiesDataset
from MIDS_script import generate_model
from my_graphs_dataset import GraphDataset
from torch_geometric.loader import DataLoader

In [ ]:
def load_dataset(root):
    # Set up parameters.
    seed = 42
    selected_graph_sizes = {
        "03-25_mix_750": -1,
    }
    split = (0.6, 0.2)
    batch_size = 1

    # Get the dataset.
    loader = GraphDataset(selection=selected_graph_sizes, seed=seed)
    prob_dataset = MIDSProbabilitiesDataset(root / "Dataset", loader)
    labels_dataset = MIDSLabelsDataset(root / "Dataset", loader, selected_extra_feature="predicted_probability")

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    prob_dataset, perm = prob_dataset.shuffle(return_perm=True)
    labels_dataset = labels_dataset.index_select(perm) # type: ignore
    assert isinstance(prob_dataset, MIDSDataset)
    assert isinstance(labels_dataset, MIDSDataset)

    # Flexible dataset splitting. Can be split to train/test or train/val/test.
    if isinstance(split, tuple):
        train_size, val_size = split
        train_size = round(train_size * len(prob_dataset))
        val_size = round(val_size * len(prob_dataset))
    else:
        train_size = round(split * len(prob_dataset))
        val_size = len(prob_dataset) - train_size

    test_probs = prob_dataset[train_size + val_size:]
    test_labels = labels_dataset[train_size + val_size:]

    # Batch and load data.
    prob_loader = DataLoader(test_probs, batch_size, shuffle=False)  # type: ignore
    labels_loader = DataLoader(test_labels, batch_size, shuffle=False)  # type: ignore
    prob_data = [test_batch for test_batch in prob_loader]
    labels_data = [test_batch for test_batch in labels_loader]

    return prob_data, labels_data, labels_dataset.num_features

### Definitions for testing functions

In [ ]:
def run_GNN(root, prob_data, label_data, num_features):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load the probability model.
    prob_model = torch.load(root / "Models" / "prob_model_best.pth")
    prob_model.to(device)
    prob_model.eval()

    # Load the best results csv file.
    load_path = root / "Results" / "labels_all_best.csv"
    best_df = pd.read_csv(load_path, index_col=0)
    selected_df = best_df[best_df['selected_extra_feature'] == 'predicted_probability']

    # Perform the experiment over models and data examples.
    records = []
    for _, row in tqdm(selected_df.iterrows(), total=len(selected_df)):
        # Load the model.
        model_kwargs = {}
        if row["architecture"] == "GIN":
            model_kwargs = {"train_eps": True}
        elif row["architecture"] == "GAT":
            model_kwargs = {"v2": True}

        label_model = generate_model(
            row["architecture"],
            num_features,
            row["hidden_channels"],
            row["gnn_layers"],
            act=row["activation"],
            jk=row["jk"] if row["jk"] != "none" else None,
            **model_kwargs
        )
        saved_model_dict = torch.load(root / "Models" / f"{row['id']}_best_model.pth", weights_only=False)
        label_model.load_state_dict(saved_model_dict["model_state_dict"])
        label_model.to(device)
        label_model.eval()

        # Calculate the execution time on each data example
        for i in trange(len(prob_data), leave=False, desc=row["architecture"]):
            start = time.perf_counter()
            # First, predict probabilities.
            example = prob_data[i].to(device)
            out = prob_model(example.x, example.edge_index)

            # Second, predict labels.
            example = label_data[i].to(device)
            out = label_model(example.x, example.edge_index)

            end = time.perf_counter()
            records.append({
                "model": row["architecture"],
                "num_nodes": example.num_nodes,
                "num_edges": example.num_edges,
                "execution_time": (end - start) * 1000
            })

    return pd.DataFrame(records)

In [ ]:
from Utilities.baselines import optimize_Gurobi

def run_Gurobi(label_data):
    records = []

    for i in trange(len(label_data)):
        nx_graph = tg_utils.to_networkx(label_data[i], to_undirected=True)
        sol, execution_time, details = optimize_Gurobi(nx_graph, "MIDS", goal="D", outputFlag=0, single_cpu=False)
        records.append({
            "model": "Gurobi",
            "num_nodes": nx_graph.number_of_nodes(),
            "num_edges": nx_graph.number_of_edges(),
            "execution_time": execution_time
        })

    return pd.DataFrame(records)

In [ ]:
from Utilities.mids_utils import find_MIDS

def run_BK(label_data):
    records = []

    for i in trange(len(label_data)):
        nx_graph = tg_utils.to_networkx(label_data[i], to_undirected=True)
        start = time.perf_counter()
        sol = find_MIDS(nx_graph)
        end = time.perf_counter()
        records.append({
            "model": "Bron-Kerbosch",
            "num_nodes": nx_graph.number_of_nodes(),
            "num_edges": nx_graph.number_of_edges(),
            "execution_time": (end - start) * 1000
        })

    return pd.DataFrame(records)

### Definitions for processing functions

In [ ]:
def process_results(results):
    # Average and standard deviation of execution time for each model over all data examples.
    overall = results.groupby("model")
    overall_avg = overall["execution_time"].mean()
    overall_std = overall["execution_time"].std()

    # Average and standard deviation of execution time for each model over all data examples with the same number of nodes.
    by_nodesize = results.groupby(["model", "num_nodes"])
    by_nodesize_avg = by_nodesize["execution_time"].mean()
    by_nodesize_std = by_nodesize["execution_time"].std()

    return (pd.DataFrame({"avg": overall_avg, "std": overall_std}),
            pd.DataFrame({"avg": by_nodesize_avg, "std": by_nodesize_std}))


def final_analysis(overall_results, by_nodesize_results):
    # Join overall results
    overall_results = pd.concat(overall_results, axis=0, keys=overall_results.keys())
    overall_results = overall_results.droplevel(0)
    print(overall_results)

    # Join by_nodesize results
    by_nodesize_results = pd.concat(by_nodesize_results, axis=0, keys=by_nodesize_results.keys())

    # Reset index for easier manipulation
    by_nodesize_results = by_nodesize_results.reset_index()
    by_nodesize_results = by_nodesize_results.drop(columns="level_0")

    # Pivot the data to create a matrix for the heatmap
    heatmap_data = by_nodesize_results.pivot_table(
        index="num_nodes", columns=["model"], values="avg"
    )

    # Add hover data for "avg" and "std"
    hover_data = []
    for model in by_nodesize_results["model"].unique():
        for size in by_nodesize_results["num_nodes"].unique():
            avg_time = by_nodesize_results[
                (by_nodesize_results["model"] == model) & (by_nodesize_results["num_nodes"] == size)
            ]["avg"].values[0]
            std_dev = by_nodesize_results[
                (by_nodesize_results["model"] == model) & (by_nodesize_results["num_nodes"] == size)
            ]["std"].values[0]
            hover_data.append((size, model, f"{avg_time:.3f} ± {std_dev:.3f}"))

    hover_df = pd.DataFrame(hover_data, columns=["num_nodes", "model", "hover"])
    hover_matrix = hover_df.pivot(index="num_nodes", columns="model", values="hover")
    hover_matrix = hover_matrix.loc[:, list(heatmap_data.columns)]

    # Plot the heatmap
    fig = px.imshow(
        heatmap_data,
        title="Average Execution Time by Node Size and Model",
        labels={"color": "Execution Time (ms)"},
        color_continuous_scale=px.colors.sequential.Viridis[::-1]
    )

    # Add custom hover template
    fig.update_traces(text=hover_matrix.values, hovertemplate="%{text}")
    fig.update_layout(
        xaxis=dict(tickmode='array', tickvals=heatmap_data.columns, ticktext=heatmap_data.columns),
        yaxis=dict(tickmode='array', tickvals=heatmap_data.index, ticktext=heatmap_data.index)
    )

    fig.show()
    fig.write_html("heatmap.html")

    return overall_results, by_nodesize_results


def make_latex_table(overall_results, by_nodesize_results):
    # Remove index from overall_results
    overall_results = overall_results.reset_index()

    # Combine avg and std columns into a single column with ± sign
    overall_results["execution_time"] = overall_results["avg"].round(3).astype(str) + " ± " + overall_results["std"].round(3).astype(str)
    overall_results = overall_results.drop(columns=["avg", "std"])
    overall_results = overall_results.rename(columns={"execution_time": "Execution Time (ms)", "model": "Method"})

    # Sort rows with a specified order
    model_order = ["Gurobi", "Bron-Kerbosch", "MLP", "GCN", "GIN", "GraphSAGE", "GAT", "GATLinNet"]
    overall_results["Method"] = pd.Categorical(overall_results["Method"], categories=model_order, ordered=True)
    overall_results = overall_results.sort_values("Method")

    # Generate LaTeX table for overall results
    overall_latex = overall_results.to_latex(index=False, header=True, column_format="l|l", escape=False)

    return overall_latex

### Run experiments

In [ ]:
root = pathlib.Path().cwd()  # For Jupyter notebook.

# Load the dataset.
prob_data, label_data, num_features = load_dataset(root)

In [ ]:
overall_results = dict()
by_nodesize_results = dict()

# Experiment for GNN models
print("Running GNN models")
GNN_results = run_GNN(root, prob_data, label_data, num_features)
overall_results["GNN"], by_nodesize_results["GNN"] = process_results(GNN_results)

# Experiment for Gurobi
print("Running Gurobi")
Gurobi_results = run_Gurobi(label_data)
overall_results["Gurobi"], by_nodesize_results["Gurobi"] = process_results(Gurobi_results)

# Repeat the experiment for Bron-Kerbosch
print("Running Bron-Kerbosch")
BK_results = run_BK(label_data)
overall_results["BK"], by_nodesize_results["BK"] = process_results(BK_results)

# Repeat the experiment for something else

### Analyze results

In [ ]:
# Analyze the results
overall_concat, by_nodesize_concat = final_analysis(overall_results, by_nodesize_results)

# Make a latex table
overall_latex = make_latex_table(overall_concat, by_nodesize_concat)
print(overall_latex)

# Save the results
overall_concat.to_csv(root / "Results" / "overall_execution_time.csv")
by_nodesize_concat.to_csv(root / "Results" / "by_nodesize_execution_time.csv")
